In [1]:
import requests

In [2]:
host = "localhost"

In [3]:
requests.get(f'http://{host}:9200/_cluster/health').json()

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 8,
 'active_shards': 8,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 80.0}

In [4]:
print(requests.get(f'http://{host}:9200/_cat/indices').text)

yellow open bioasq                         wyN4IWYjRx6GZ4SjauW4Ww 1 1 1490  0  24.8mb  24.8mb
green  open .kibana-event-log-7.9.3-000001 jEUEvzSLQgitqDIoaT86-A 1 0    6  0  32.3kb  32.3kb
green  open .apm-custom-link               7iyymmzcSgSMzpAoDSNjdw 1 0    0  0    208b    208b
green  open .kibana_task_manager_1         WfXRIDU7SPW_ThUCtCXhRQ 1 0    6 14 140.3kb 140.3kb
green  open .apm-agent-configuration       opSZfOBRTW24pzv_b-7tmA 1 0    0  0    208b    208b
yellow open label                          Sby2XbYJRISi9VgpGlpmIQ 1 1    0  0    208b    208b
green  open .kibana_1                      k4cQ2vThSKmaVZDM5RXgbg 1 0   31 11  10.4mb  10.4mb



### Initialize ElasticsearchDocumentStore

In [5]:
from haystack.document_stores import ElasticsearchDocumentStore

/home/ec2-user/anaconda3/envs/dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
doc_store = ElasticsearchDocumentStore(
    host=host,
    username='',
    password='',
    index='bioasq'
)

### Load documents from S3

In [7]:
import boto3
import json

In [8]:
s3 = boto3.resource('s3')

In [9]:
bucket = 'domain-qa-system'
key = 'elasticsearch/ingestion/bioasq/bio_v1.json'
obj = s3.Object(bucket, key)

In [10]:
docs = json.load(obj.get()['Body']) 

### Write documents to ES

In [ ]:
doc_store.delete_all_documents()

In [11]:
doc_store.write_documents(docs)

In [12]:
requests.get(f'http://{host}:9200/bioasq/_count').json()

{'count': 1490,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}